# Create the random DPI Networks (rand perturbomes)
1.) Define Thresholds  
2.) Calculate Thresholds (take original results/take random resuls)  
3.) Get significant interactions  
4.) Save results   
5.) Compare with real results

In [ ]:
import numpy as np
from matplotlib import pylab as plt
import networkx as nx
import os
import random

## 1. Define Thresholds
Define which thresholds to be used, as well as which wells should be excluded (e.g. Cytotoxic Drugs)

In [23]:


# Certain wells show specific problem that need to be excluded from further analysis
problematic_Well_Signs = ['ProblemWithCombinationWell','CombinationTransferProblem','Drug1Problem','Drug2Problem','CytotoxicDrug1','CytotoxicDrug2','CytotoxicCombination']

#Types of possible interactions
interactionTypes = ['Emergent','Increasing','Decreasing']

#Batches of the screen
batches = ['Batch1','Batch2']


use_original_drugpairs = True

'''
#############
Thresholds###
#############
'''

#Choose the parameter to find an optimal balance between interactions significance and effect size
interaction_significance = 3 #how far away in means of mahalanobis distances from the NI point cloud does the real interaction need to be to be significant
perturbaion_significnace = 7 #how far away in means of mahalanobis distances from DMSO does a perturbation need to be to be significanctly perturbed

AlphaBeta_MAD_range = 2 #range of normal perturbed alpha/beta values per drug for non signifant drug pairs
Gamma_percentile = 100 #range of normal perturbed gamma values per drug for non signifant drug pairs

## 2. Calculate Thresholds
This can either be based on the original drug pairs (standard) or the random pairs

In [24]:
# Some Easy Outlier detection
def reject_outliers_2(data, m=6.):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d / (mdev if mdev else 1.)
    #return s < m
    return [data[i] for i in range(0, len(data)) if s[i] < m]


In [25]:
#Open the file
if use_original_drugpairs:
    fp = open('../data/Create_DPI_Network/All_MC_Scores.csv','r')
else:
    fp = open('../data/Create_DPI_Network_Random/All_MC_Scores.csv','r')
fp.next()

#File to save all non-significant alpha/beta/gamma values (to later calculate suited thresholds)
perDrug_AlphaBetaGammas = {'Batch1':{},'Batch2':{}}

#collect all alpha/beta/gammas
for line in fp:
    tmp = line.strip().split(',')
    
    #ignore the drug pair if there seemed to be a problem
    if tmp[5] in problematic_Well_Signs:
        continue
    
    #extract drug and batch information
    drug1 = tmp[0]
    drug2 = tmp[1]
    batch = tmp[2]
    
    #add the drug to the dictionary if not already so
    if perDrug_AlphaBetaGammas[batch].has_key(drug1) == False:
        perDrug_AlphaBetaGammas[batch][drug1] = {'AlphaBetas':[],'Gammas':[]}
    if perDrug_AlphaBetaGammas[batch].has_key(drug2) == False:
        perDrug_AlphaBetaGammas[batch][drug2] = {'AlphaBetas':[],'Gammas':[]}
    
    #get the interaction significance
    maha_General = float(tmp[21])
    #mp_General = float(tmp[22])

    #include only non-significant interactions
    if  maha_General < interaction_significance:
        
        #add the alpha/beta values
        perDrug_AlphaBetaGammas[batch][drug1]['AlphaBetas'].append(float(tmp[14]))
        perDrug_AlphaBetaGammas[batch][drug2]['AlphaBetas'].append(float(tmp[15]))
        
        #add the same gamma to BOTH drugs (as gamma is undirected)
        perDrug_AlphaBetaGammas[batch][drug1]['Gammas'].append(float(tmp[16]))
        perDrug_AlphaBetaGammas[batch][drug2]['Gammas'].append(float(tmp[16]))
fp.close()
        
#Dictionary with the thresholds for each drug        
perDrug_AlphaBetaGammaThresholds = {'Batch1':{},'Batch2':{}}       

#calculate thresholds
for b in batches:
    for key in perDrug_AlphaBetaGammas[b]:

        #create alpha/beta threshold by using the borders of alpha/beta within 2 MADs and gamma within 99 percentiles
        alphaBeta_NoOutlier = reject_outliers_2(perDrug_AlphaBetaGammas[b][key]['AlphaBetas'], AlphaBeta_MAD_range)
        alphaBeta_NoOutlier.append(1)
        perDrug_AlphaBetaGammas[b][key]['Gammas'].append(0)
        perDrug_AlphaBetaGammaThresholds[b][key] = {'Upper':max([x for x in alphaBeta_NoOutlier if x >= 1]), 'Lower':min([x for x in alphaBeta_NoOutlier if x <= 1]), 'Emergent':np.percentile(perDrug_AlphaBetaGammas[b][key]['Gammas'],Gamma_percentile)}
        


## 3. Get Significant Interactions
Go through all drug pairs and associate significant interactions (effect/sgnificance) correctly to interactions according to our vector math e.g. 1.2 alpha would be a increasing interaction. 

In [26]:
#How many interactions could be in the end calculated i.e. after problematic/non working drugs / combinatons were removed
Number_Of_Real_Combinations = sum(1 for line in open('../data/Create_DPI_Network/All_MC_Scores.csv'))

print 'Number of real drug pairs: %d' %Number_Of_Real_Combinations

#Get all random drug pairs in one list
fp = open('../data/Create_DPI_Network_Random/All_MC_Scores.csv','r')
fp.next()

# add all random pairs calculated (approx. 300k) of which randomly 30k are picked
RandomDrugPairs = []
for line in fp:
    RandomDrugPairs.append(line)

Number of real drug pairs: 29827


In [50]:
#Save number of individual interaction type for each random network creation
number_significant_pairs = []
number_interactions = []
number_increasing = []
number_decreasing = []
number_emergent = []

#Create 10k random DPIs (i.e. randomly pick 10k times 30k pairs)
for i in range(0,10000):
    
    significant_Drugs = {'Batch1':set(),'Batch2':set()}

    InteractionCount = 0
    Number_Of_Valid_DrugPairs = 0
    Interactions = {}
    
    for iT in interactionTypes:
        Interactions[iT] = {}

    #get one random assemble of random lines
    random_combinations = random.sample(lines,Number_Of_Real_Combinations)
    
    for line in random_combinations:
        tmp = line.strip().split(',')
        Number_Of_Valid_DrugPairs +=1

        if tmp[7] in problematic_Well_Signs:
            continue

        #get the interaction significance
        maha_General = float(tmp[21])
        
        #only check interaction if significantly away from NI point
        if maha_General > interaction_significance:

            #increment significant interaction count
            InteractionCount += 1

            #extract drug information
            drug1 = tmp[0]
            drug2 = tmp[1]
            batch = tmp[3]

            #extract mahalanobis distances (how far away from DMSO)
            maha_drug1 = float(tmp[6])
            maha_drug2 = float(tmp[9])
            maha_Combi = float(tmp[12])

            #check for significance (usually > 6)
            drug1_significance = maha_drug1 > perturbaion_significnace
            drug2_significance = maha_drug2 > perturbaion_significnace
            combi_significance = maha_Combi > perturbaion_significnace

            #extract alpha/beta/gamma
            alpha = float(tmp[15])
            beta = float(tmp[16])
            gamma =  float(tmp[17])
            
            #add drugs to significant drugs if significant
            if drug1_significance:
                significant_Drugs[batch].add(drug1)
            if drug2_significance:
                significant_Drugs[batch].add(drug2)

            #####
            # GOT THROUGH ALL 7 POSSIBILITIES of where a drug can be modulated
            ###
            #0.
            # Singles not active, combination not active ==> no interactions

            # 1.)
            #Single not active, combination active ==> possible Emergent
            if drug1_significance == False and drug2_significance == False and combi_significance == True:
                #if gamma > Emergent_Threshold[batch]:
                if gamma > max([perDrug_AlphaBetaGammaThresholds[batch][drug1]['Emergent'],perDrug_AlphaBetaGammaThresholds[batch][drug2]['Emergent']]):
                    Interactions['Emergent'][drug1+','+drug2] = {'Value':gamma,'Batch':batch, 'Mahalanobis':maha_General}
            #2.)
            #One Single active, combination active anymore ==> possible Deactivting
            elif drug1_significance == True and drug2_significance == False and combi_significance == False:
                #if alpha < AlphaBeta_Threshold_Decreasing[batch]:
                if alpha < perDrug_AlphaBetaGammaThresholds[batch][drug1]['Lower']:
                    Interactions['Decreasing'][drug2+','+drug1] = {'Value':alpha,'Batch':batch, 'Mahalanobis':maha_General}
            #3.)
            #One Single active, combination active anymore ==> possible Deactivting
            elif drug1_significance == False and drug2_significance == True and combi_significance == False:
                #if beta < AlphaBeta_Threshold_Decreasing[batch]:
                if beta < perDrug_AlphaBetaGammaThresholds[batch][drug2]['Lower']:
                    Interactions['Decreasing'][drug1+','+drug2] = {'Value':beta,'Batch':batch, 'Mahalanobis':maha_General}
            #4.)
            #Both Single active, combination not active anymore ==> possible Double Deactivating
            elif drug1_significance == True and drug2_significance == True and combi_significance == False:
                #if beta < AlphaBeta_Threshold_Decreasing[batch]:
                if beta < perDrug_AlphaBetaGammaThresholds[batch][drug2]['Lower']:
                    Interactions['Decreasing'][drug1+','+drug2] = {'Value':beta,'Batch':batch, 'Mahalanobis':maha_General}

                if alpha < perDrug_AlphaBetaGammaThresholds[batch][drug1]['Lower']:
                    Interactions['Decreasing'][drug2+','+drug1] = {'Value':alpha,'Batch':batch, 'Mahalanobis':maha_General}

            #5.)
            #One Single active, combination active  ==> The one Single gets modulated
            elif drug1_significance == True and drug2_significance == False and combi_significance == True:
                #print tmp
                #if gamma > Emergent_Threshold[batch]:
                if gamma > max([perDrug_AlphaBetaGammaThresholds[batch][drug1]['Emergent'],perDrug_AlphaBetaGammaThresholds[batch][drug2]['Emergent']]):
                    Interactions['Emergent'][drug1+','+drug2] = {'Value':gamma,'Batch':batch, 'Mahalanobis':maha_General}

                if alpha > perDrug_AlphaBetaGammaThresholds[batch][drug1]['Upper']:
                    Interactions['Increasing'][drug2+','+drug1] = {'Value':alpha,'Batch':batch , 'Mahalanobis':maha_General}

                if alpha < perDrug_AlphaBetaGammaThresholds[batch][drug1]['Lower']:
                    Interactions['Decreasing'][drug2+','+drug1] = {'Value':alpha,'Batch':batch , 'Mahalanobis':maha_General}
            #6.)
            #One Single active, combination active  ==> The one Single gets modulated
            elif drug1_significance == False and drug2_significance == True and combi_significance == True:

                #if gamma > Emergent_Threshold[batch]:
                if gamma > max([perDrug_AlphaBetaGammaThresholds[batch][drug1]['Emergent'],perDrug_AlphaBetaGammaThresholds[batch][drug2]['Emergent']]):
                    Interactions['Emergent'][drug1+','+drug2] = {'Value':gamma,'Batch':batch, 'Mahalanobis':maha_General}
                if beta > perDrug_AlphaBetaGammaThresholds[batch][drug2]['Upper']:
                    Interactions['Increasing'][drug1+','+drug2] = {'Value':beta,'Batch':batch , 'Mahalanobis':maha_General}
                if beta < perDrug_AlphaBetaGammaThresholds[batch][drug2]['Lower']:
                    Interactions['Decreasing'][drug1+','+drug2] = {'Value':beta,'Batch':batch , 'Mahalanobis':maha_General}
            #7.)
            #Both Single active, combination active  ==> Both Single gets modulated
            elif drug1_significance == True and drug2_significance == True and combi_significance == True:
                if gamma > max([perDrug_AlphaBetaGammaThresholds[batch][drug1]['Emergent'],perDrug_AlphaBetaGammaThresholds[batch][drug2]['Emergent']]):
                    Interactions['Emergent'][drug1+','+drug2] = {'Value':gamma,'Batch':batch, 'Mahalanobis':maha_General}


                if alpha > perDrug_AlphaBetaGammaThresholds[batch][drug1]['Upper']:
                    Interactions['Increasing'][drug2+','+drug1] = {'Value':alpha,'Batch':batch , 'Mahalanobis':maha_General}
                if alpha < perDrug_AlphaBetaGammaThresholds[batch][drug1]['Lower']:
                    Interactions['Decreasing'][drug2+','+drug1] = {'Value':alpha,'Batch':batch , 'Mahalanobis':maha_General}


                if beta > perDrug_AlphaBetaGammaThresholds[batch][drug2]['Upper']:
                    Interactions['Increasing'][drug1+','+drug2] = {'Value':beta,'Batch':batch, 'Mahalanobis':maha_General }
                if beta < perDrug_AlphaBetaGammaThresholds[batch][drug2]['Lower']:
                    Interactions['Decreasing'][drug1+','+drug2] = {'Value':beta,'Batch':batch, 'Mahalanobis':maha_General }
            
    
    NumberOfInteractions = 0      
    for iT in interactionTypes:
    #print 'Number %s: %d' %(iT , len(Interactions[iT]))
        NumberOfInteractions = NumberOfInteractions + len(Interactions[iT])
    
    
    number_significant_pairs.append(InteractionCount)
    number_interactions.append(NumberOfInteractions)
    number_increasing.append(len(Interactions['Increasing']))
    number_decreasing.append(len(Interactions['Decreasing']))
    number_emergent.append(len(Interactions['Emergent']))



## 4. Save results
Create a result file

In [55]:
#Print all results
fp_out = open('../results/Create_DPI_Network_Random/Results.csv','w')
fp_out.write('Type,Counts\n')
fp_out.write('NumberSignificantPairs,'+';'.join([str(x) for x in number_significant_pairs])+'\n')
fp_out.write('NumberInteractionss,'+';'.join([str(x) for x in number_interactions])+'\n')
fp_out.write('NumberIncreasing,'+';'.join([str(x) for x in number_increasing])+'\n')
fp_out.write('NumberDecerasing,'+';'.join([str(x) for x in number_decreasing])+'\n')
fp_out.write('NumberEmergent,'+';'.join([str(x) for x in number_emergent])+'\n')
fp_out.close()

## 5. Compare with real results
Create plots and calulcate Z-Score showing the difference between the 10k randomly created DPI networks and the real DPI network (perturbome)

In [51]:
DPI = nx.read_gml('../data/Create_DPI_Network_Random/DPI_Network_Complete.gml')

real_number_significant_pairs = 11961

real_number_interactions = len(DPI.edges())

interactions = []
for edge in list(set(DPI.edges())):
    for key in DPI[edge[0]][edge[1]]:
        interactions.append(DPI[edge[0]][edge[1]][key]['Type'])
        
real_number_increasing = interactions.count('Increasing')
real_number_decreasing = interactions.count('Decreasing')
real_number_emergent = interactions.count('Emergent')

print 'Real Results:'
print 'Number Significant pairs: %d' %real_number_significant_pairs
print 'Number Interactions: %d' %real_number_interactions
print 'Number Increasing: %d' %real_number_increasing
print 'Number Decreasing: %d' %real_number_decreasing
print 'Number Emergent: %d' %real_number_emergent

Real Results:
Number Significant pairs: 11961
Number Interactions: 1832
Number Increasing: 446
Number Decreasing: 814
Number Emergent: 572


In [75]:

# Number Significant Pairs
ZScore = (real_number_significant_pairs - np.mean(number_significant_pairs))/np.std(number_significant_pairs)
plt.title('Number Significant Pairs')
plt.hist(number_significant_pairs,bins=30, color='#969696', edgecolor='#969696', linewidth=1.2)
plt.axvline(real_number_significant_pairs, ls='--', c ='#40B9D4')
plt.legend(['ZScore: %.2f' %ZScore])
#plt.show()
plt.savefig('../results/Create_DPI_Network_Random/Number_Significant_Pairs.pdf')
plt.close()

# Number Interactions
ZScore = (real_number_interactions - np.mean(number_interactions))/np.std(number_interactions)
plt.title('Number Interactions')
plt.hist(number_interactions,bins=30, color='#969696', edgecolor='#969696', linewidth=1.2)
plt.axvline(real_number_interactions, ls='--', c ='grey')
plt.legend(['ZScore: %.2f' %ZScore])
#plt.show()
plt.savefig('../results/Create_DPI_Network_Random/Number_Interactions.pdf')
plt.close()

# Number Decreasing
ZScore = (real_number_decreasing - np.mean(number_decreasing))/np.std(number_decreasing)
plt.title('Number Decreasing')
plt.hist(number_decreasing,bins=30, color='#969696', edgecolor='#969696', linewidth=1.2)
plt.axvline(real_number_decreasing, ls='--', c ='#F70020')
plt.legend(['ZScore: %.2f' %ZScore])
#plt.show()
plt.savefig('../results/Create_DPI_Network_Random/Number_Decreasing.pdf')
plt.close()

# Number Decreasing
ZScore = (real_number_increasing - np.mean(number_increasing))/np.std(number_increasing)
plt.title('Number Increasing')
plt.hist(number_increasing,bins=28, color='#969696', edgecolor='#969696', linewidth=1.2)
plt.axvline(real_number_increasing, ls='--', c ='#ACD900')
plt.legend(['ZScore: %.2f' %ZScore])
#plt.show()
plt.savefig('../results/Create_DPI_Network_Random/Number_Increasing.pdf')
plt.close()

# Number Emergent
ZScore = (real_number_emergent - np.mean(number_emergent))/np.std(number_emergent)
plt.title('Number Emergent')
plt.hist(number_emergent,bins=30, color='#969696', edgecolor='#969696', linewidth=1.2)
plt.axvline(real_number_emergent, ls='--', c ='#0096FF')
plt.legend(['ZScore: %.2f' %ZScore])
#plt.show()
plt.savefig('../results/Create_DPI_Network_Random/Number_Emergent.pdf')
plt.close()